In Shopify, we support two types of relative pagination: using a since_id and following URLs from the Link header in the response. In the following sections we’ll give examples of how to use each one and explain the tradeoffs of both.

#### Pagination with `since_id`

This is the simplest form of relative cursor pagination. Each request will include a `since_id` parameter with the id of the last record from the previous page. This does require the records to be sorted by id ascending, but this happens automatically when a `since_id` parameter is present. To ensure the first page is sorted by id, you can include a `since_id` parameter with a value of 0. This form of pagination already existed on all endpoints prior to the 2019-07 API version.

For example, if you’re requesting pages of products from your shop, the request for the first page might look like the following:

https://shop-domain.myshopify.com/admin/products.json?limit=5&fields=id,title&since_id=0

In [126]:
import requests
import json
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999


total_results = []
page_num = 1
username = '7e86d45b24d9a3da5da72a93a8210f34'
password = '3ddc548cff03ffd7d064c7e2fa7f3431'
shop = 'paulas-choice-singapore'
api_version = '2020-04'
resource = 'orders'
since_id = 2277025054785


url = f'https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=50&since_id={since_id}'
page_info = ['dummy']

while len(page_info) > 0 :   
    print(f"requesting page {page_num}: {url}")
#     print(next_page)
    result = requests.get(url, auth=(username, password))
#     print(result.headers)
    data = result.json()
    total_results = total_results + data['orders']
    result_headers = result.headers
    page_info = re.findall(r', <(.+?)>(?=; rel="next")', str(result_headers))
    print("page_info: \n",page_info)
    print(len(page_info))
    if len(page_info) > 0:
        url = page_info[0] 
        print("overriden url: ", url)
#         print("page_info :", page_info[0])
    else: break
    page_num += 1


requesting page 1: https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=50&since_id=2277025054785
page_info: 
 ['https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=50&page_info=eyJvcmRlciI6ImlkIGFzYyIsImxhc3RfaWQiOjIyODE3MDU2MDMxMzcsImxhc3RfdmFsdWUiOiIyMjgxNzA1NjAzMTM3IiwiZGlyZWN0aW9uIjoibmV4dCJ9']
1
overriden url:  https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=50&page_info=eyJvcmRlciI6ImlkIGFzYyIsImxhc3RfaWQiOjIyODE3MDU2MDMxMzcsImxhc3RfdmFsdWUiOiIyMjgxNzA1NjAzMTM3IiwiZGlyZWN0aW9uIjoibmV4dCJ9
requesting page 2: https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=50&page_info=eyJvcmRlciI6ImlkIGFzYyIsImxhc3RfaWQiOjIyODE3MDU2MDMxMzcsImxhc3RfdmFsdWUiOiIyMjgxNzA1NjAzMTM3IiwiZGlyZWN0aW9uIjoibmV4dCJ9
page_info: 
 ['https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=50&page_info=eyJkaXJlY3Rpb24iOiJuZXh0Iiwib3JkZXIiOiJpZCBhc2MiLCJsYXN0X2lkI

In [130]:
print(total_results)

# pd.DataFrame.from_dict(total_results)
with open('since_2277025054785_191569954497.json', 'w') as file:
    json.dump(total_results, file)

[{'id': 2277076926529, 'email': 'alex.nguyen814108@gmail.com', 'closed_at': None, 'created_at': '2020-06-14T20:55:58+08:00', 'updated_at': '2020-06-14T20:56:00+08:00', 'number': 53501, 'note': None, 'token': '0d094f27bb5f3f843d7164e563ec45ad', 'gateway': 'stripe', 'test': False, 'total_price': '78.40', 'subtotal_price': '78.40', 'total_weight': 153, 'total_tax': '0.00', 'taxes_included': False, 'currency': 'SGD', 'financial_status': 'paid', 'confirmed': True, 'total_discounts': '0.00', 'total_line_items_price': '78.40', 'cart_token': '83018232aa70a8adb48b6c27000f268c', 'buyer_accepts_marketing': True, 'name': '191569954501', 'referring_site': '', 'landing_site': "/collections/25th-anniversary-special-oily-combi?utm_campaign=Anniversary 2020 Oily/Combi Reminder (TTKjiL)&utm_medium=email&utm_source=Paula's Choice Subscribers&_ke=eyJrbF9lbWFpbCI6ICJhbGV4Lm5ndXllbjgxNDEwOEBnbWFpbC5jb20iLCAia2xfY29tcGFueV9pZCI6ICJMY1lrU3YifQ==", 'cancelled_at': None, 'cancel_reason': None, 'total_price_usd'

In [129]:
# print(f'requesting page {page_num}: {url}')
# result = requests.get(url, auth=(username, password))
# data = result.json()
# total_results = total_results + data['orders']
# next_page = ["dummy"] # header from first page already has "previous" link, dict() function can't be used
# next_page = dict(result.headers)['Link'].split(";")[0][1:-1]
# print("next_page :\n", next_page)


# Single page query
# url = f'https://{shop}.myshopify.com/admin/api/{api_version}/{resource}.json?limit=20'
# result = requests.get(url, auth=(username, password))
# data = result.json()
# total_results = total_results + data['orders']
# print(dict(result.headers)['Link'])
# page_num += 1
# print(f"last_page has value {last_page}")

# url_page_info = f'https://{shop}.myshopify.com/admin/api/{api_version}/{resource}.json?limit=50&page_info={page_info}'

In [ ]:
# next_page
# re.findall(r', <(.+?)>(?=; rel="next"\', )',str(requests.get(next_page, auth=(username, password)).headers))
page_1 = 'https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=50&since_id=2277025054785'
page_2 = 'https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=50&page_info=eyJvcmRlciI6ImlkIGFzYyIsImxhc3RfaWQiOjIyNzcwNzY5MjY1MjksImxhc3RfdmFsdWUiOiIyMjc3MDc2OTI2NTI5IiwiZGlyZWN0aW9uIjoicHJldiJ9'
page_3 = 'https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=50&page_info=eyJvcmRlciI6ImlkIGFzYyIsImxhc3RfaWQiOjIyODE3MDU2MDMxMzcsImxhc3RfdmFsdWUiOiIyMjgxNzA1NjAzMTM3IiwiZGlyZWN0aW9uIjoibmV4dCJ9'
result = requests.get(page_1, auth=(username, password))
# result = requests.get(page_info[0], auth=(username, password))
print(result.headers)
# next_page = dict(result.headers)['Link'].split(";")[0][1:-1]
# print(next_page, "\n\n")
print(re.findall(r', <(.+?)>(?=; rel="next")', str(result.headers)))

In [109]:
# page_info = re.findall(r', <(.+?)>(?=; rel="next")', str(result.headers))
# page_info
result.headers

{'Date': 'Thu, 18 Jun 2020 03:19:34 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=db95c66a51ca4a7e1a9b89d963fd916d81592450373; expires=Sat, 18-Jul-20 03:19:33 GMT; path=/; domain=.myshopify.com; HttpOnly; SameSite=Lax', 'X-Sorting-Hat-PodId': '64', 'X-Sorting-Hat-ShopId': '14402440', 'Vary': 'Accept-Encoding', 'Referrer-Policy': 'origin-when-cross-origin', 'X-Frame-Options': 'DENY', 'X-ShopId': '14402440', 'X-ShardId': '64', 'X-Stats-UserId': '', 'X-Stats-ApiClientId': '1481447', 'X-Stats-ApiPermissionId': '34963423', 'X-Shopify-API-Terms': 'By accessing or using the Shopify API you agree to the Shopify API License and Terms of Use at https://www.shopify.com/legal/api-terms', 'HTTP_X_SHOPIFY_SHOP_API_CALL_LIMIT': '1/80', 'X-Shopify-Shop-Api-Call-Limit': '1/80', 'X-Shopify-API-Version': '2020-04', 'Link': '<https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=50&pa

### Single Order Query

In [76]:
url = 'https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders/2277025054785.json'
result = requests.get(url, auth=(username, password))
data = result.json()
data

{'order': {'id': 2277025054785,
  'email': 'hayley.hm.cheng@gmail.com',
  'closed_at': '2020-06-17T00:15:17+08:00',
  'created_at': '2020-06-14T20:36:54+08:00',
  'updated_at': '2020-06-17T00:15:17+08:00',
  'number': 53497,
  'note': None,
  'token': '33eaf893e986c12c5c5920251819e006',
  'gateway': 'stripe',
  'test': False,
  'total_price': '45.00',
  'subtotal_price': '45.00',
  'total_weight': 76,
  'total_tax': '0.00',
  'taxes_included': False,
  'currency': 'SGD',
  'financial_status': 'paid',
  'confirmed': True,
  'total_discounts': '0.00',
  'total_line_items_price': '45.00',
  'cart_token': 'c05c078013e4959c549a0279ac735486',
  'buyer_accepts_marketing': True,
  'name': '191569954497',
  'referring_site': '',
  'landing_site': '/',
  'cancelled_at': None,
  'cancel_reason': None,
  'total_price_usd': '32.31',
  'checkout_token': 'bf9a8a95c2c64d78ef2ef0187c324b1a',
  'reference': None,
  'user_id': None,
  'location_id': None,
  'source_identifier': None,
  'source_url': None

In [78]:
pd.DataFrame.from_dict(total_results)

,id,email,closed_at,created_at,updated_at,number,note,token,gateway,test,total_price,subtotal_price,total_weight,total_tax,taxes_included,currency,financial_status,confirmed,total_discounts,total_line_items_price,cart_token,buyer_accepts_marketing,name,referring_site,landing_site,cancelled_at,cancel_reason,total_price_usd,checkout_token,reference,user_id,location_id,source_identifier,source_url,processed_at,device_id,phone,customer_locale,app_id,browser_ip,landing_site_ref,order_number,discount_applications,discount_codes,note_attributes,payment_gateway_names,processing_method,checkout_id,source_name,fulfillment_status,tax_lines,tags,contact_email,order_status_url,presentment_currency,total_line_items_price_set,total_discounts_set,total_shipping_price_set,subtotal_price_set,total_price_set,total_tax_set,line_items,fulfillments,refunds,total_tip_received,original_total_duties_set,current_total_duties_set,admin_graphql_api_id,shipping_lines,billing_address,shipping_address,client_details,payment_details,customer
0,2277076926529,alex.nguyen814108@gmail.com,None,2020-06-14T20:55:58+08:00,2020-06-14T20:56:00+08:00,53501,None,0d094f27bb5f3f843d7164e563ec45ad,stripe,False,78.40,78.40,153,0.00,False,SGD,paid,True,0.00,78.40,83018232aa70a8adb48b6c27000f268c,True,191569954501,,/collections/25th-anniversary-special-oily-com...,None,None,56.30,de214821238cce9dbd407e48dd0d17ec,None,None,None,None,None,2020-06-14T20:55:57+08:00,None,None,en,580111,101.127.1.5,None,54501,[],[],[],[stripe],direct,12928104169537,web,None,[],,alex.nguyen814108@gmail.com,https://paulaschoice.sg/14402440/orders/0d094f...,SGD,"{'shop_money': {'amount': '78.40', 'currency_c...","{'shop_money': {'amount': '0.00', 'currency_co...","{'shop_money': {'amount': '0.00', 'currency_co...","{'shop_money': {'amount': '78.40', 'currency_c...","{'shop_money': {'amount': '78.40', 'currency_c...","{'shop_money': {'amount': '0.00', 'currency_co...","[{'id': 4914065571905, 'variant_id': 277534585...",[],[],0.0,None,None,gid://shopify/Order/2277076926529,"[{'id': 1859882811457, 'title': 'Free Delivery...","{'first_name': 'Alex', 'address1': '490 Admira...","{'first_name': 'Alex', 'address1': '490 Admira...","{'browser_ip': '101.127.1.5', 'accept_language...","{'credit_card_bin': '426588', 'avs_result_code...","{'id': 395186602049, 'email': 'alex.nguyen8141..."
1,2277141610561,samanthalim79@hotmail.com,None,2020-06-14T21:19:18+08:00,2020-06-14T21:19:23+08:00,53511,None,8c68b9549bcadbff66a5888696037832,stripe,False,226.00,226.00,174,0.00,False,SGD,paid,True,0.00,226.00,1fda54dadc39d7d6b8fc4a96444ba028,True,191569954511,,/collections/25th-anniversary-special-oily-com...,None,None,162.28,429742e23028723b0bceffa5ca8cc9f6,None,None,None,None,None,2020-06-14T21:19:18+08:00,None,None,en,580111,121.6.20.55,None,54511,[],[],[],[stripe],direct,12928189956161,web,None,[],,samanthalim79@hotmail.com,https://paulaschoice.sg/14402440/orders/8c68b9...,SGD,"{'shop_money': {'amount': '226.00', 'currency_...","{'shop_money': {'amount': '0.00', 'currency_co...","{'shop_money': {'amount': '0.00', 'currency_co...","{'shop_money': {'amount': '226.00', 'currency_...","{'shop_money': {'amount': '226.00', 'currency_...","{'shop_money': {'amount': '0.00', 'currency_co...","[{'id': 4914202705985, 'variant_id': 921365102...",[],[],0.0,None,None,gid://shopify/Order/2277141610561,"[{'id': 1859945168961, 'title': 'Free Delivery...","{'first_name': 'Samantha', 'address1': '241 We...","{'first_name': 'Samantha', 'address1': '241 We...","{'browser_ip': '121.6.20.55', 'accept_language...","{'credit_card_bin': '418238', 'avs_result_code...","{'id': 2685386162241, 'email': 'samanthalim79@..."
2,2278816153665,bearpig80@gmail.com,None,2020-06-15T12:39:55+08:00,2020-06-15T12:40:02+08:00,53570,None,dfa01a05f044b228bb8e772927d98f93,stripe,False,29.00,29.00,159,0.00,False,SGD,paid,True,10.00,39.00,d4ef35648780e1a2e61626b199ec54ee,True,191569954570,,/,None,None,20.82,7f7e7cea0e24049a42d8552cdc7a0a39,None,None,None,None,None,202

In [21]:
df_result = pd.DataFrame.from_dict(result.json()['orders'])
df_result[['created_at', 'updated_at']] = df_result[['created_at', 'updated_at']].apply(pd.to_datetime)

In [28]:
df_result

,id,email,closed_at,created_at,updated_at,number,note,token,gateway,test,total_price,subtotal_price,total_weight,total_tax,taxes_included,currency,financial_status,confirmed,total_discounts,total_line_items_price,cart_token,buyer_accepts_marketing,name,referring_site,landing_site,cancelled_at,cancel_reason,total_price_usd,checkout_token,reference,user_id,location_id,source_identifier,source_url,processed_at,device_id,phone,customer_locale,app_id,browser_ip,landing_site_ref,order_number,discount_applications,discount_codes,note_attributes,payment_gateway_names,processing_method,checkout_id,source_name,fulfillment_status,tax_lines,tags,contact_email,order_status_url,presentment_currency,total_line_items_price_set,total_discounts_set,total_shipping_price_set,subtotal_price_set,total_price_set,total_tax_set,line_items,fulfillments,refunds,total_tip_received,original_total_duties_set,current_total_duties_set,admin_graphql_api_id,shipping_lines,billing_address,shipping_address,client_details,payment_details,customer
0,2268605710401,nurshahidah.ithin@gmail.com,None,2020-06-11 12:29:31+08:00,2020-06-11 12:29:42+08:00,52696,None,e599a44bbb3cb78a4eae764c5523ab97,stripe,False,130.00,130.00,88,0.00,False,SGD,paid,True,0.00,130.00,c1f47214bc94967bb0d41a2046f95c91,True,191569953696,https://www.google.com/,/,None,None,93.52,eb82982939036c54e641c52aa43cdda3,None,None,None,None,None,2020-06-11T12:29:30+08:00,None,+6591597034,en,580111,116.89.91.196,None,53696,[],[],[],[stripe],direct,12909945290817,web,None,[],,nurshahidah.ithin@gmail.com,https://paulaschoice.sg/14402440/orders/e599a4...,SGD,"{'shop_money': {'amount': '130.00', 'currency_...","{'shop_money': {'amount': '0.00', 'currency_co...","{'shop_money': {'amount': '0.00', 'currency_co...","{'shop_money': {'amount': '130.00', 'currency_...","{'shop_money': {'amount': '130.00', 'currency_...","{'shop_money': {'amount': '0.00', 'currency_co...","[{'id': 4897352384577, 'variant_id': 277537557...",[],[],0.0,None,None,gid://shopify/Order/2268605710401,"[{'id': 1852555231297, 'title': 'Free Delivery...","{'first_name': 'shahidah', 'address1': '922 Ta...","{'first_name': 'shahidah', 'address1': '922 Ta...","{'browser_ip': '116.89.91.196', 'accept_langua...","{'credit_card_bin': '526471', 'avs_result_code...","{'id': 2846852808769, 'email': 'nurshahidah.it..."
1,2268559573057,rani.sidhu@etonhouse.edu.sg,None,2020-06-11 11:54:22+08:00,2020-06-11 11:54:29+08:00,52695,None,c295e923d9daceed68a4c896aca1f9e1,stripe,False,255.00,255.00,422,0.00,False,SGD,paid,True,0.00,255.00,e71b281642ca6b5925da036e0c72f53e,False,191569953695,https://www.google.com.sg/,/,None,None,183.44,7b496ef535680f2ca4a718aca9e123ea,None,None,None,None,None,2020-06-11T11:54:21+08:00,None,None,en,580111,116.87.19.124,None,53695,[],[],[],[stripe],direct,12909813596225,web,None,[],,rani.sidhu@etonhouse.edu.sg,https://paulaschoice.sg/14402440/orders/c295e9...,SGD,"{'shop_money': {'amount': '255.00', 'currency_...","{'shop_money': {'amount': '0.00', 'currency_co...","{'shop_money': {'amount': '0.00', 'currency_co...","{'shop_money': {'amount': '255.00', 'currency_...","{'shop_money': {'amount': '255.00', 'currency_...","{'shop_money': {'amount': '0.00', 'currency_co...","[{'id': 4897255817281, 'variant_id': 277531297...",[],[],0.0,None,None,gid://shopify/Order/2268559573057,"[{'id': 1852517449793, 'title': 'Free Delivery...","{'first_name': 'Rani s', 'address1': 'Blk 451 ...","{'first_name': 'Rani s', 'address1': 'Blk 451 ...","{'browser_ip': '116.87.19.124', 'accept_langua...","{'credit_card_bin': '526471', 'avs_result_code...","{'id': 3086731673665, 'email': 'rani.sidhu@eto..."
2,2268557344833,sjyunrou@yahoo.com.sg,None,2020-06-11 11:52:49+08:00,2020-06-11 11:53:00+08:00,52694,None,819fdb62e073f208db176a1628f57cbc,stripe,False,14.00,14.00,159,0.00,False,SGD,paid,True,25.00,39.00,19720c3b6db19ba7d8f612541f0054e2,True,191569953694,https://www.google.com/,/,None,None,10.07,cb24c2a7716bd538d6d1765d85c389f0,None,None,None,None,None,2020-06

In [5]:
pd.Series(result.headers)

Date                                                      Thu, 11 Jun 2020 04:57:28 GMT
Content-Type                                            application/json; charset=utf-8
Transfer-Encoding                                                               chunked
Connection                                                                   keep-alive
Set-Cookie                            __cfduid=d16a52bead7e023da639e7be38809faa91591...
X-Sorting-Hat-PodId                                                                  64
X-Sorting-Hat-ShopId                                                           14402440
Vary                                                                    Accept-Encoding
Referrer-Policy                                                origin-when-cross-origin
X-Frame-Options                                                                    DENY
X-ShopId                                                                       14402440
X-ShardId                       

In [31]:
dict(result.headers)

{'Date': 'Thu, 11 Jun 2020 04:53:28 GMT',
 'Content-Type': 'application/json; charset=utf-8',
 'Transfer-Encoding': 'chunked',
 'Connection': 'keep-alive',
 'Set-Cookie': '__cfduid=d5f02fc629f2aa4635103623352cc93761591851207; expires=Sat, 11-Jul-20 04:53:27 GMT; path=/; domain=.myshopify.com; HttpOnly; SameSite=Lax',
 'X-Sorting-Hat-PodId': '64',
 'X-Sorting-Hat-ShopId': '14402440',
 'Vary': 'Accept-Encoding',
 'Referrer-Policy': 'origin-when-cross-origin',
 'X-Frame-Options': 'DENY',
 'X-ShopId': '14402440',
 'X-ShardId': '64',
 'X-Stats-UserId': '',
 'X-Stats-ApiClientId': '1481447',
 'X-Stats-ApiPermissionId': '34963423',
 'X-Shopify-API-Terms': 'By accessing or using the Shopify API you agree to the Shopify API License and Terms of Use at https://www.shopify.com/legal/api-terms',
 'HTTP_X_SHOPIFY_SHOP_API_CALL_LIMIT': '1/80',
 'X-Shopify-Shop-Api-Call-Limit': '1/80',
 'X-Shopify-API-Version': '2020-04',
 'Strict-Transport-Security': 'max-age=7889238',
 'X-Request-Id': 'f4f4aa9d-a46

In [21]:
pd.Series(result.headers)

Date                                                      Thu, 11 Jun 2020 04:53:00 GMT
Content-Type                                            application/json; charset=utf-8
Transfer-Encoding                                                               chunked
Connection                                                                   keep-alive
Set-Cookie                            __cfduid=da66a95c1b5520faa08c55c4b579446511591...
X-Sorting-Hat-PodId                                                                  64
X-Sorting-Hat-ShopId                                                           14402440
Vary                                                                    Accept-Encoding
Referrer-Policy                                                origin-when-cross-origin
X-Frame-Options                                                                    DENY
X-ShopId                                                                       14402440
X-ShardId                       

In [5]:
result.headers['Link']

'<https://paulas-choice-singapore.myshopify.com/admin/api/2020-04/orders.json?limit=20&page_info=eyJsYXN0X2lkIjoyMjY2MjIxODcxMTY5LCJsYXN0X3ZhbHVlIjoiMjAyMC0wNi0xMCAwNDowNToyMyIsImRpcmVjdGlvbiI6Im5leHQifQ>; rel="next"'

In [14]:
result.json()['orders']

[{'id': 2266326204481,
  'email': 'chris.wanling@gmail.com',
  'closed_at': None,
  'created_at': '2020-06-10T13:48:35+08:00',
  'updated_at': '2020-06-10T13:48:40+08:00',
  'number': 52648,
  'note': None,
  'token': 'e07f0dc13a1406a2b2d72d8c50d53fe8',
  'gateway': 'stripe',
  'test': False,
  'total_price': '45.00',
  'subtotal_price': '45.00',
  'total_weight': 158,
  'total_tax': '0.00',
  'taxes_included': False,
  'currency': 'SGD',
  'financial_status': 'paid',
  'confirmed': True,
  'total_discounts': '0.00',
  'total_line_items_price': '45.00',
  'cart_token': 'e77f4cceef45ec0bf9def53f02c710d6',
  'buyer_accepts_marketing': False,
  'name': '191569953648',
  'referring_site': '',
  'landing_site': '/',
  'cancelled_at': None,
  'cancel_reason': None,
  'total_price_usd': '32.43',
  'checkout_token': '204fc2ce162af5cea4becff13efc1079',
  'reference': None,
  'user_id': None,
  'location_id': None,
  'source_identifier': None,
  'source_url': None,
  'processed_at': '2020-06-10T

## BQ Insert Script

In [ ]:
import json
import requests
from google.cloud import bigquery

client = bigquery.Client()

def trade_gecko_to_bq(request):
    params = request.get_json(force=True)
    
    if not 'project' in params or not 'dataset' in params or not 'table' in params:
        return "Missing project, BigQuery dataset or table argument!"
        
    bq_table = params['project'] + '.' + params['dataset'] + '.' + params['table']
    
    try:
        client.get_table(bq_table)
    except:
        # Most likely table is non-existent
        return "Project, BigQuery dataset or table non-existent!"    
    
    url = 'https://api.tradegecko.com/variants'
    token = 'f4855aebc4a92c0d6a09f07b105bcbae81afbaf8cb1344f47a5b5c45cf8f4c1e'
    result = get_and_insert(bq_table, url, token)
    
    return json.dumps(result)

        
def get_and_insert(bq_table, url, token):
    
    bearer = {'Authorization': f'Bearer {token}'}
    limit_num = 250
    page_num = 1
 
    total_records = 0
    fail_records_to_insert = 0
    result = { 'status': 'success' }
    page_loop = True
    while page_loop:
        url = f'{url}?limit={limit_num}&page={page_num}'
        resp = requests.get(url, headers=bearer)
        if resp.status_code == 200:
            json_data = resp.json()
            
            if 'variants' in json_data:
                insert_responses = insert_to_bq(bq_table, json_data['variants'])
                
                if 'inserts' not in result:
                    result['inserts'] = []
                result['inserts'].append({ 'page': page_num, 'insert_responses': insert_responses })
                
                pagination = json.loads(resp.headers['X-Pagination'])
                total_records = pagination['total_records']
                page_loop = not pagination['last_page']
                page_num += 1
                fail_records_to_insert += len(insert_responses)
            else:
                result['status'] = 'failure'
                result['message'] = 'No "variants" attribute in text returned by Trade Gecko HTTP response'
                return result
        else:
            result['status'] = 'failure'
            result['message'] = 'Trade Gecko HTTP Response not 200 status code'
            return result
    
    result['total_records'] = f'{total_records - fail_records_to_insert} of {total_records} successfully inserted to BigQuery table `{bq_table}`'
    return result
                
                
def insert_to_bq(bq_table, data):
    for variant in data:
        variant['upc'] = None if variant['upc'] == '' else variant['upc']
        variant['stock_levels'] = convert_to_list(variant['stock_levels'], 'stock_id', 'level')
        variant['committed_stock_levels'] = convert_to_list(variant['committed_stock_levels'], 'stock_id', 'level')
        variant['prices'] = convert_to_list(variant['prices'], 'type', 'value')

    return client.insert_rows_json(table=bq_table, json_rows=data)
    

def convert_to_list(json_dict, key_label, value_label):
    if len(json_dict) == 0:
        return []
    else:
        return [{key_label: k, value_label: json_dict[k]} for k in json_dict]
